In [1]:
import duckdb
import pandas

In [2]:
import faker
import uuid
import random, time, datetime
import time

In [4]:
fake = faker.Faker()

In [3]:

device = ["desktop", "mobil", "Tablet"]
weights_device = [0.4, 0.4, 0.2]
link = {"ИПОТЕКА": 1, "ОСАГО": 2, "ДОМ": 3, "ДМС": 4, "КАСКО": 50}
weights_link = [0.1, 0.4, 0.1, 0.15, 0.25]
weights_action = [0.6, 0.3, 0.1]

In [5]:
data_list = []
for i in range(10):
    session_start = datetime.datetime.now()
    session_end = datetime.datetime.now() + datetime.timedelta(milliseconds=random.randint(1000, 1000000))
    name_link = random.choices(list(link.keys()), weights=weights_link)[0] if (session_end - session_start).seconds > 15 else None
    action_type = random.choices(['calculation', 'view', 'add to cart'], weights=weights_action)[0] if name_link is not None else 'view'
    data = {"user_id": int(random.randint(100, 500000) * (time.time() * 1000 % 10)),
            "session_id": str(uuid.uuid4()).split('-')[0],
            "traffic_source": random.choice(['organic', 'paid', 'social', 'email', 'referral', 'direct']),
            "session_start": session_start,
            "session_end": session_end,
            "device": random.choices(device, weights=weights_device)[0],
            "name_link": name_link,
            "action_type": action_type,
            "purchase_amount": round(random.uniform(1000 * (link[name_link] / 2), 10000 * link[name_link]), 2) if (action_type == "calculation" and name_link is not None) else None,
            }
    data_list.append(data)

In [6]:
df = pandas.DataFrame(data_list)
duckdb.sql("from df").show()    

┌─────────┬────────────┬────────────────┬────────────────────────────┬────────────────────────────┬─────────┬───────────┬─────────────┬─────────────────┐
│ user_id │ session_id │ traffic_source │       session_start        │        session_end         │ device  │ name_link │ action_type │ purchase_amount │
│  int64  │  varchar   │    varchar     │        timestamp_ns        │        timestamp_ns        │ varchar │  varchar  │   varchar   │     double      │
├─────────┼────────────┼────────────────┼────────────────────────────┼────────────────────────────┼─────────┼───────────┼─────────────┼─────────────────┤
│ 1785805 │ 01f14ae7   │ referral       │ 2025-07-16 19:48:04.938891 │ 2025-07-16 19:51:22.556178 │ desktop │ ДМС       │ calculation │        35148.81 │
│  674342 │ 4a5ae122   │ referral       │ 2025-07-16 19:48:04.939303 │ 2025-07-16 19:56:00.332305 │ desktop │ ОСАГО     │ calculation │         2396.85 │
│ 3806535 │ e4bd6bb4   │ direct         │ 2025-07-16 19:48:04.939356 │ 2025-

In [15]:
conn_str = (
    "host=0.0.0.0 "
    "port=5433 "
    "dbname=analytical_db "
    "user=postgres "
    "password=postgres"
)

In [18]:
duckdb.sql(f""" ATTACH '{conn_str}' AS postgres_db (TYPE postgres);
                 """)

In [19]:
duckdb.sql(f""" insert into postgres_db.stg_analytical.action_users
                select * from df; """)